If you want to follow the following notebook, it is not enough with following the steps described at "README for fine-tuning", you need to install autogluon following the instructions on its web: https://auto.gluon.ai/stable/install.html

In [1]:
import datasets
from gluonts.dataset.arrow import ArrowWriter
from pathlib import Path
from typing import Any

d:\apps\envs\LoRAenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_training_chronos_format(dataset: datasets.arrow_dataset.Dataset, H: int | Any = None):
    return dataset.map(
        lambda ts: 
                {'start': str(ts['timestamp'][0]), 
                 'target': ts['target'][:-H]
                 }, 
                 remove_columns=['id', 'timestamp'])

In [3]:
path = "autogluon/chronos_datasets"
datasets_with_horizons = [
                {'name': 'monash_cif_2016', 'H': 12},
                {'name': 'monash_car_parts', 'H': 12},
                {'name': 'monash_hospital', 'H': 12},
                {'name': 'monash_fred_md', 'H': 12},
                {'name': 'exchange_rate', 'H': 30}
                ]
datasets_for_finetuning = [datasets.load_dataset(path=path, name=ds['name'], split='train') for ds in datasets_with_horizons]
datasets_for_finetuning

[Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 72
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 2674
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 767
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 107
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 8
 })]

In [4]:
for ds in datasets_for_finetuning:
    ds.set_format('numpy')

In [ ]:
expected_schema = datasets.Features({
    "start": datasets.Value("string"),
    "target": datasets.Sequence(datasets.Value("float64"))
})

In [5]:
datasets_for_finetuning = list(map(
    lambda ds, ds_conf: to_training_chronos_format(ds, ds_conf['H']), 
    datasets_for_finetuning, 
    datasets_with_horizons))
datasets_for_finetuning

[Dataset({
     features: ['target', 'start'],
     num_rows: 72
 }),
 Dataset({
     features: ['target', 'start'],
     num_rows: 2674
 }),
 Dataset({
     features: ['target', 'start'],
     num_rows: 767
 }),
 Dataset({
     features: ['target', 'start'],
     num_rows: 107
 }),
 Dataset({
     features: ['target', 'start'],
     num_rows: 8
 })]

In [ ]:
#datasets_for_finetuning = datasets.concatenate_datasets(datasets_for_finetuning)
datasets_for_finetuning

In [7]:
for ds, ds_conf in zip(datasets_for_finetuning, datasets_with_horizons):
    ArrowWriter(compression="lz4").write_to_file(
        ds,
        path=Path.cwd()/ "training" / "data" / f"{ds_conf['name']}.arrow",
    )

In [ ]:
!python training/train.py --config ./training/configs/chronos-t5-small-lora.yaml

In [ ]:
datasets_for_finetuning[0][0]

In [ ]:
import pandas as pd
prueba = pd.DataFrame({'item_id': [0 for i in range(10)], 
                       'timestamp': ['1970-01-31', '1970-02-28', '1970-03-31', '1970-04-30', '1970-05-31', 
                                     '1972-04-30', '1972-05-31', '1972-06-30', '1972-07-31', '1972-08-31'], 
                       'target': [76, 54, 32,56,43,11,37,41,75, 90]})
prueba

In [ ]:
prueba = TimeSeriesDataFrame.from_data_frame(prueba)
prueba

In [ ]:
train, test = prueba.train_test_split(3)

In [ ]:
train

In [ ]:
test